In [1]:
import sys
#sys.path.append('C:/Pradap/Research/Python-Packages/enrique/')
sys.path.append('/scratch/pradap/python-work/enrique/')

In [2]:
import jpype

In [3]:
jpype.getDefaultJVMPath()

'/scratch/pradap/local/share/jdk1.7.0_25/jre/lib/amd64/server/libjvm.so'

In [14]:
import magellan as mg

In [ ]:
mg.init_jvm

In [5]:
#i/o
A = mg.read_csv('../magellan/datasets/table_A.csv', key = 'ID')
B = mg.read_csv('../magellan/datasets/table_B.csv', key = 'ID')

In [6]:
k = A.values

In [7]:
import numpy as np
np.delete(k, 0, 1)

array([['Kevin Smith', 1989, 30.0, '607 From St, San Francisco', 94107],
       ['Michael Franklin', 1988, 27.5, '1652 Stockton St, San Francisco',
        94122],
       ['William Bridge', 1986, 32.0, '3131 Webster St, San Francisco',
        94107],
       ['Binto George', 1987, 32.5, '423 Powell St, San Francisco', 94122],
       ['Alphonse Kemper', 1984, 35.0, '1702 Post Street, San Francisco',
        94122]], dtype=object)

In [8]:
A['birth_year']

0    1989
1    1988
2    1986
3    1987
4    1984
Name: birth_year, dtype: int64

In [9]:
import readline
for i in range(readline.get_current_history_length()):
    print readline.get_history_item(i)

In [11]:
%edit 1-4

IPython will make a temporary file named: /tmp/ipython_edit_1uUVh0/ipython_edit_jON8TJ.py


In [13]:
type(%hist)

SyntaxError: invalid syntax (<ipython-input-13-dc87e336291f>, line 1)